In [1]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
#import os
#os.environ["TF_USE_LEGACY_KERAS"] = "True"


In [2]:
#!pip uninstall tensorflow tensorflow-macos tensorflow-metal -y
#!pip uninstall tensorflow -y
#!pip install --upgrade tensorflow


In [3]:
#!pip install tensorflow==2.13.0
#!pip install tensorflow
#!pip install tf_keras


In [4]:
#pip show tensorflow
#!pip show tensorflow

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, Normalizer
import pandas_ta as ta
from pandas.tseries.offsets import MonthEnd, BDay, Week
from itertools import product
from sklearn.metrics import mean_absolute_error

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from datetime import datetime, timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
#from skopt import BayesSearchCV

from skopt.space import Real, Integer, Categorical


from tensorflow.keras.models import Sequential

#from tensorflow.keras.optimizers.legacy import Adam as LegacyAdam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from skopt import gp_minimize

from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization

import tensorflow as tf
#from tensorflow.keras.optimizers.legacy import Adam
import time
#from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.optimizers.legacy import RMSprop
from tensorflow.keras.optimizers.legacy import Adam, SGD

#from keras.models import Sequential
#from keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Bidirectional, concatenate, Add
    

In [6]:
# Function for loading ETF data
def data_loading(ticker_symbol, start_date, end_date):
    etf_data = yf.Ticker(ticker_symbol)
    etf_history = etf_data.history(start=start_date, end=end_date)
    etf_history.index = etf_history.index.tz_localize(None)
    return etf_history, etf_data

#Function to derive the predictor columns
def etf_predictors(etf_history,  start_date, end_date, etf_data, benchmark_ticker = '^GSPC' ):
    
    # Calculate Daily Returns
    etf_history['Daily Return'] = etf_history['Close'].pct_change()

    # Calculate 21-Day Volatility (standard deviation of daily returns, annualized)
    etf_history['Volatility'] = etf_history['Daily Return'].rolling(window=21).std() * np.sqrt(252)
    #etf_history['Volatility_ta'] = ta.volatility(etf_history['Close'], window=21, annualize=True)
   
    # Get the ETF info
    etf_info = etf_data.info

    # Retrieve the net asset value price (NAV) and total net assets
    nav_price = etf_info['navPrice']
    total_assets = etf_info['totalAssets']

    # Calculate the number of shares outstanding
    shares_outstanding = total_assets / nav_price

    # Calculate Market Capitalization for each day
    # Market Capitalization = Closing Price * Total Number of Shares Outstanding
    etf_history['Market Cap'] = etf_history['Close'] * shares_outstanding

    total_assets = etf_info['totalAssets']
    total_liabilities = etf_info.get('totalLiabilities', 0)  # Handle the case where total liabilities might not be present
    
    # Calculate Book Value per Share
    book_value_per_share = (total_assets - total_liabilities) / shares_outstanding
    
    # Calculate Price to Book (P/B) Ratio
    etf_history['P/B Ratio'] = etf_history['Close'] / book_value_per_share
    
    # Calculate 1-Month Momentum (21 trading days)
    
    etf_history['Momentum'] = ta.mom(etf_history['Close'], length=21)
    
    benchmark_data = yf.download(benchmark_ticker, start=start_date, end=end_date)
    benchmark_data['Daily Return'] = benchmark_data['Close'].pct_change()
    #benchmark_data_history = benchmark_data.dropna()
    benchmark_data.head(10)
    
    # Ensure the indices are time zone-naive
    benchmark_data.index = benchmark_data.index.tz_localize(None)
    combined_data = etf_history[['Close']].join(benchmark_data[['Close']], lsuffix='_ETF', rsuffix='_Benchmark')
    combined_data = combined_data.dropna()
    
    # Calculate rolling beta with a 30-day window
    rolling_beta_21 = rolling_beta(combined_data, window=21)

    # Add the rolling beta to the dataframe
    combined_data.loc[:, 'Rolling Beta 21-day'] = rolling_beta_21
    etf_history['Rolling Beta']=combined_data['Rolling Beta 21-day']
    
    # Calculate daily profitability
    daily_profitabilities = []
    previous_nav = None
    for index, row in etf_history.iterrows():
        current_nav = row['Close']  # Current day's NAV
        #print(current_nav)
        if previous_nav is not None:
            daily_profitability = (current_nav - previous_nav) / previous_nav * 100
            daily_profitabilities.append(daily_profitability)
        else:
            daily_profitabilities.append(None)
        previous_nav = current_nav  # Update previous_nav for the next iteration

    # Add daily profitabilities to ETF dataset
    etf_history['Daily Profitability (%)'] = daily_profitabilities
    
    # Calculate the dividend yield for each day
    dividend_yields = []

    for index, row in etf_history.iterrows():
        # Get the dividend payment for the day
        dividend_payment = row['Dividends']

        # Get the current market price of the ETF for the day
        current_price = row['Close']

        # Calculate the dividend yield for the day
        dividend_yield = (dividend_payment / current_price) * 100

        # Append the dividend yield to the list
        dividend_yields.append(dividend_yield)
      
    etf_history['Div yield'] = dividend_yields

    #Volatility
    etf_history['ATR'] = ta.atr(etf_history['High'], etf_history['Low'], etf_history['Close'], length=21)    
    
    # Compute the Relative Volatility Index (RVI)
    rvi = ta.rvi(etf_history['Close'],length=21)
    etf_history['RVI'] = rvi
    
    #Momentum
    rsi_window = 14  # Window size for RSI calculation
    roc_window = 12  # Window size for ROC calculation
    #rsi_window = 21  
    #roc_window = 21  

    

    etf_history['RSI'] = ta.rsi(etf_history['Close'],length=rsi_window)
    # Calculate Rate of Change (ROC)
    etf_history['ROC'] = ta.roc(etf_history['Close'], length=roc_window)
        
    etf_history['log_returns'] = np.log(etf_history['Close'] / etf_history['Close'].shift(1))

    return etf_history

# Function to calculate rolling beta
def rolling_beta(df, window):
    rolling_cov = df['Close_Benchmark'].rolling(window=window).cov(df['Close_ETF'])
    rolling_var = df['Close_Benchmark'].rolling(window=window).var()
    rolling_beta = rolling_cov / rolling_var
    return rolling_beta



In [7]:
def pre_processing(
    etf_history, 
    train_start_date,
    train_end_date,
    test_start_date,
    test_end_date,
    prediction_dates_monthly,
    prediction_dates_weekly,
    feature_columns=None,
    scaling_strategy='StandardScaler', 
    final_end_date='2025-01-01'
):
    """
    Preprocess ETF history data and prepare train/test splits along with 
    dynamically scaled forecast data for both weekly & monthly horizons,
    stored in separate dictionaries.

    Parameters
    ----------
    etf_history : pd.DataFrame
        Full historical data with at least 'log_returns' column plus other features.
    train_start_date : str
        Start date (YYYY-MM-DD) for training data.
    train_end_date : str
        End date (YYYY-MM-DD) for training data.
    test_start_date : str
        Start date (YYYY-MM-DD) for test data.
    test_end_date : str
        End date (YYYY-MM-DD) for test data.
    prediction_dates_monthly : list of str
        List of start dates for each monthly forecast window.
    prediction_dates_weekly : list of str
        List of start dates for each weekly forecast window.
    feature_columns : list of str, optional
        Columns to be used as features. If None, defaults are used.
    scaling_strategy : str, optional
        'StandardScaler' or 'Normalizer'. Defaults to 'StandardScaler'.
    final_end_date : str, optional
        The final end date (YYYY-MM-DD) for the last forecast window.
        Defaults to '2024-12-01'.

    Returns
    -------
    dict
        A dictionary containing:
        - 'train_data_scaled': np.ndarray
        - 'test_data_scaled': np.ndarray
        - 'y_train': pd.Series
        - 'y_test': pd.Series
        - 'scaler': scaler object
        - 'X': pd.DataFrame (all feature columns before splitting)
        - 'etf_history': pd.DataFrame (original data with missing values handled)
        - 'forecast_data_weekly': dict of weekly forecast DataFrame slices
        - 'forecast_data_monthly': dict of monthly forecast DataFrame slices
        - 'forecast_data_weekly_scaled': dict of scaled weekly forecast slices
        - 'forecast_data_monthly_scaled': dict of scaled monthly forecast slices
    """

    # 1. Handle missing values and infinite values
    etf_history.fillna(etf_history.median(), inplace=True)
    etf_history.replace([np.inf, -np.inf], 0, inplace=True)

    # 2. Default feature columns if none are provided
    if feature_columns is None:
        feature_columns = [
            'Volatility', 'Volume', 'Daily Return', 'Market Cap', 
            'P/B Ratio', 'Momentum', 'Rolling Beta', 
            'Daily Profitability (%)', 'ATR', 'RVI', 'RSI', 'ROC'
        ]

    # 3. Select features (X) and target (y)
    X = etf_history.loc[:, feature_columns]
    y = etf_history['log_returns']

    # 4. Split train and test data
    train_data = X.loc[train_start_date : train_end_date]
    test_data  = X.loc[test_start_date  : test_end_date]
    y_train = y.loc[train_start_date : train_end_date]
    y_test  = y.loc[test_start_date  : test_end_date]

    # 5. Initialize and fit scaler
    if scaling_strategy == 'StandardScaler':
        scaler = StandardScaler()
    elif scaling_strategy == 'Normalizer':
        scaler = Normalizer()
    else:
        raise ValueError(f"Unsupported scaling strategy: {scaling_strategy}")

    train_data_scaled = scaler.fit_transform(train_data)
    test_data_scaled  = scaler.transform(test_data)

    # 6. Create separate dictionaries for weekly and monthly
    forecast_data_weekly = {}
    forecast_data_monthly = {}

    # ---- Weekly Forecast Slices ----
    for i, start_date in enumerate(prediction_dates_weekly):
        if i < len(prediction_dates_weekly) - 1:
            weekly_end = (pd.to_datetime(start_date) + Week(1) - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
        else:
            weekly_end = final_end_date
        
        w_key = f'forecast_data_{i+1}w'
        forecast_data_weekly[w_key] = X.loc[start_date : weekly_end]

    # ---- Monthly Forecast Slices ----
    for i, start_date in enumerate(prediction_dates_monthly):
        if i < len(prediction_dates_monthly) - 1:
            monthly_end = (pd.to_datetime(start_date) + MonthEnd(0)).strftime('%Y-%m-%d')
        else:
            monthly_end = final_end_date
        
        m_key = f'forecast_data_{i+1}m'
        forecast_data_monthly[m_key] = X.loc[start_date : monthly_end]

    # 7. Scale forecast data (weekly & monthly) and add intercept
    forecast_data_weekly_scaled = {}
    for period_key, period_data in forecast_data_weekly.items():
        if not period_data.empty:
            data_scaled = scaler.transform(period_data)
            forecast_data_weekly_scaled[period_key] = sm.add_constant(data_scaled)

    forecast_data_monthly_scaled = {}
    for period_key, period_data in forecast_data_monthly.items():
        if not period_data.empty:
            data_scaled = scaler.transform(period_data)
            forecast_data_monthly_scaled[period_key] = sm.add_constant(data_scaled)

    # 8. Add a constant to train and test data
    train_data_scaled = sm.add_constant(train_data_scaled)
    test_data_scaled  = sm.add_constant(test_data_scaled)

    # 9. Return all objects in a dictionary
    return {
        'train_data_scaled': train_data_scaled,
        'test_data_scaled': test_data_scaled,
        'y_train': y_train,
        'y_test': y_test,
        'scaler': scaler,
        'X': X,
        'etf_history': etf_history,
        'forecast_data_weekly': forecast_data_weekly,
        'forecast_data_monthly': forecast_data_monthly,
        'forecast_data_weekly_scaled': forecast_data_weekly_scaled,
        'forecast_data_monthly_scaled': forecast_data_monthly_scaled
    }


In [8]:
def build_and_train_lstm_model(epochs, batch_size, units, dropout_rate, learning_rate, train_data_scaled, y_train, validation_split=0.2):
    
    train_data_reshaped = reshape_for_lstm(train_data_scaled)
    
    # Input layer
    input_layer = Input(shape=(train_data_reshaped.shape[1], train_data_reshaped.shape[2]))

    # First Bidirectional LSTM layer
    bidirectional_lstm_1 = Bidirectional(LSTM(units=int(units), return_sequences=True))(input_layer)
    dropout_layer = Dropout(dropout_rate)(bidirectional_lstm_1)

    # Second Bidirectional LSTM layer
    bidirectional_lstm_2 = Bidirectional(LSTM(units=int(units / 2), return_sequences=False))(dropout_layer)
    dropout_layer_2 = Dropout(dropout_rate)(bidirectional_lstm_2)

    # First Dense hidden layer with ReLU activation
    dense_hidden = Dense(units=int(units / 2), activation='relu')(dropout_layer_2)
    dropout_layer_3 = Dropout(dropout_rate)(dense_hidden)

    # Second Dense hidden layer
    dense_hidden_2 = Dense(units=int(units / 4), activation='relu')(dropout_layer_3)
    dropout_layer_4 = Dropout(dropout_rate)(dense_hidden_2)

    # Output layer
    output_layer = Dense(1, activation='linear')(dropout_layer_4)

    # Build the model
    model = Model(inputs=input_layer, outputs=output_layer)
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    # Early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Train the model
    model.fit(train_data_reshaped, y_train, 
              epochs=int(epochs), 
              batch_size=int(batch_size), 
              validation_split=validation_split, 
              callbacks=[early_stopping],
              verbose=0)
    
    return model



def optimize_hyperparameters(train_data_scaled, y_train, test_data_scaled, y_test):
    def objective_function(epochs, batch_size, units, dropout_rate, learning_rate):
        model = build_and_train_lstm_model(epochs, batch_size, units, dropout_rate, learning_rate, train_data_scaled, y_train)
        test_data_reshaped = reshape_for_lstm(test_data_scaled)
        test_predictions = make_predictions(model, test_data_reshaped)
        mse = mean_squared_error(y_test, test_predictions)
        return -mse  # Negative MSE because we maximize in Bayesian Optimization

    # Define parameter bounds
    param_bounds = {
        'batch_size': (16, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (20, 40),
        'learning_rate': (1e-4, 5e-3),
        'units': (32, 256),
    }

    optimizer = BayesianOptimization(
        f=objective_function,
        pbounds=param_bounds,
        verbose=2,
        random_state=42,
    )

    # Run optimization
    optimizer.maximize(init_points=25, n_iter=50)  # Initial random points + iterations
    best_params = optimizer.max['params']
    return best_params


def make_predictions(model, data_scaled):
    if len(data_scaled.shape) == 2:
        data_scaled = np.reshape(data_scaled, (data_scaled.shape[0], 1, data_scaled.shape[1]))
    elif len(data_scaled.shape) != 3:
        raise ValueError(f"Unexpected shape for input data: {data_scaled.shape}")
    
    predictions = model.predict(data_scaled)
    return predictions


def eval_model(best_model, test_data_scaled, y_test, y_train=None):
    # Reshape test data to match LSTM input requirements
    test_data_reshaped = test_data_scaled.reshape((test_data_scaled.shape[0], 1, test_data_scaled.shape[1]))
    
    # Make predictions
    test_predictions = best_model.predict(test_data_reshaped)
    
    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, test_predictions)
    
    # Calculate Mean Absolute Scaled Error (MASE) - Example calculation assuming seasonal period m=1
    naive_forecast = y_test.shift(1).fillna(method='bfill')
    mase = mae / mean_absolute_error(y_test, naive_forecast)
    print("mae: ",mae)
    print("mase: ",mase)
    return mae, mase

# Function to make predictions
def predictions(model, forecast_data_scaled, original_forecast_data):
    forecast_results = {}
    for key, data in forecast_data_scaled.items():
        # Reshape data for compatibility with the LSTM model
        if len(data.shape) == 2:
            data = data.reshape((data.shape[0], 1, data.shape[1]))
        
        # Make predictions
        predictions = model.predict(data)
        
        # Use the index from the corresponding original forecast DataFrame
        original_index = original_forecast_data[key].index
        
        # Create a DataFrame for the predictions
        forecast_results[key] = pd.DataFrame(
            predictions, 
            columns=["log_returns"], 
            index=original_index
        )
    return forecast_results



def calculate_mase(y_true, y_pred, naive_forecast):
    # Calculate the MAE of the model's predictions
    mae_model = mean_absolute_error(y_true, y_pred)
    
    # Calculate the MAE of the naive forecast
    mae_naive = mean_absolute_error(y_true, naive_forecast)
    
    # Calculate MASE
    mase = mae_model / mae_naive
    return mase

def reshape_for_lstm(data):
    return np.reshape(data, (data.shape[0], 1, data.shape[1]))  # Reshape into (samples, time_steps, features)


In [9]:
def combined_workflow(
    tickers,
    start_date, 
    end_date, 
    train_start_date, 
    train_end_date, 
    test_start_date, 
    test_end_date,
    prediction_dates_monthly, 
    prediction_dates_weekly
):
    etf_results = {}
    
    for ticker in tickers:
        
        print(ticker)
        # 1. Load and preprocess data
        etf_history, etf_data = data_loading(ticker, start_date, end_date)
        etf_history = etf_predictors(etf_history, start_date, end_date, etf_data)

        # 2. Single call to pre_processing for training/testing & both forecast horizons
        preprocessed = pre_processing(
            etf_history=etf_history, 
            train_start_date=train_start_date, 
            train_end_date=train_end_date,
            test_start_date=test_start_date, 
            test_end_date=test_end_date,
            prediction_dates_monthly=prediction_dates_monthly,
            prediction_dates_weekly=prediction_dates_weekly
        )
        
        # 3. Train the model
        #model = model_training(preprocessed['train_data_scaled'], preprocessed['y_train'])
        
        # Track the start time
        start_time = time.time()


        # Optimize hyperparameters
        best_params = optimize_hyperparameters(preprocessed['train_data_scaled'], preprocessed['y_train'], preprocessed['test_data_scaled'], preprocessed['y_test'])

        print('Best Params:',best_params)
        # Extract the best hyperparameters
        best_epochs = int(best_params['epochs'])
        best_batch_size = int(best_params['batch_size'])
        best_units = int(best_params['units'])
        best_dropout_rate = best_params['dropout_rate']
        best_learning_rate = best_params['learning_rate']

        # Train the final model using the best hyperparameters
        model = build_and_train_lstm_model(
            epochs=best_epochs, 
            batch_size=best_batch_size, 
            units=best_units, 
            dropout_rate=best_dropout_rate, 
            learning_rate=best_learning_rate, 
            train_data_scaled=preprocessed['train_data_scaled'], 
            y_train=preprocessed['y_train']
        )


        # Calculate time consumed
        time_consumed = time.time() - start_time

        # Print the time consumed
        print(f"Time consumed for training: {time_consumed:.2f} seconds")
        
        # 4. Evaluate the model
        mae, mase = eval_model(
            model, 
            preprocessed['test_data_scaled'], 
            preprocessed['y_test'], 
            preprocessed['y_train']
        )
        
        # 5. Generate predictions for monthly forecasts
        monthly_forecast_results = predictions(
            model,
            preprocessed['forecast_data_monthly_scaled'],
            preprocessed['forecast_data_monthly']  # Pass unscaled forecast data for index preservation
        )
        # 6. Generate predictions for weekly forecasts

        weekly_forecast_results = predictions(
            model,
            preprocessed['forecast_data_weekly_scaled'],
            preprocessed['forecast_data_weekly']  # Pass unscaled forecast data for index preservation
        )


        # ---------------------------------------------------------------------
        # 7. Build your final dictionary in the desired format
        # ---------------------------------------------------------------------

        # A. Basic structure
        dict_data = {
            "etf_history": etf_history,              # The full ETF dataframe (with predictors, etc.)
            "X": preprocessed["X"],                 # All feature columns used
            "y_train_values": preprocessed["y_train"],  # Training target values

            # Store model performance
            "model results": {
                "mae": mae,
                "mase": mase,
                "time": time_consumed
            },
        }

        # B. Store monthly forecast predictions in separate keys
        #    Suppose you want up to 12 monthly periods; adjust as needed
        for i_m in range(1, 13):
            # The keys produced by predictions(...) might look like 'forecast_data_1m', 'forecast_data_2m', etc.
            forecast_key = f"forecast_data_{i_m}m"

            if forecast_key in monthly_forecast_results:
                # monthly_forecast_results[forecast_key] might be a DataFrame with 'log_returns'
                df_forecast = monthly_forecast_results[forecast_key]
                
                # e.g. store array of predicted 'log_returns' (or your final target variable)
                dict_data[f"forecast_predictions_df_{i_m}m"] = df_forecast["log_returns"].values
            
            # If a given month's key doesn't exist, you might skip it or handle differently

        # C. Store weekly forecast predictions similarly, if desired
        #    For example, suppose you do up to 48 weekly windows
        for i_w in range(1, 53):
            forecast_key = f"forecast_data_{i_w}w"
            if forecast_key in weekly_forecast_results:
                df_forecast = weekly_forecast_results[forecast_key]
                dict_data[f"forecast_predictions_df_{i_w}w"] = df_forecast["log_returns"].values

        # Finally, store this dictionary per ticker
        etf_results[ticker] = dict_data
    
    return etf_results


In [10]:
import pandas as pd

# Example usage
tickers = ['SMH', 'SOXX', 'PSI', 'XSD', 'IYW', 'XLK', 'VGT', 'FTEC', 'IGM', 'IXN']
start_date, end_date = '2000-01-01', '2024-12-31'
train_start_date, train_end_date, test_start_date, test_end_date = (
    start_date,         # train_start_date
    '2014-01-01',       # train_end_date
    '2014-01-01',       # test_start_date
    '2024-01-01'        # test_end_date
)

# Generate prediction dates for monthly & weekly
prediction_dates_monthly = pd.date_range(
    start='2024-01-01', 
    periods=12, 
    freq='MS'
).strftime('%Y-%m-%d').tolist()

prediction_dates_weekly = pd.date_range(
    start='2024-01-01', 
    periods=52, 
    freq='W-MON'
).strftime('%Y-%m-%d').tolist()

# Run the workflow
results = combined_workflow(
    tickers=tickers,
    start_date=start_date,
    end_date=end_date,
    train_start_date=train_start_date,
    train_end_date=train_end_date,
    test_start_date=test_start_date,
    test_end_date=test_end_date,
    prediction_dates_monthly=prediction_dates_monthly,
    prediction_dates_weekly=prediction_dates_weekly
)

# The 'results' object is a dictionary keyed by ticker.
# For each ticker (e.g. 'SMH' or 'SOXX'), to inspect:
#   results[ticker]["etf_history"]
#   results[ticker]["X"]
#   results[ticker]["y_train_values"]
#   results[ticker]["model results"]
#   results[ticker]["forecast_predictions_df_1m"], etc.
#
# Example:
# Print the first monthly forecast predictions for 'SMH'
#print(results['SMH']['forecast_predictions_df_1m'])


SMH


[*********************100%%**********************]  1 of 1 completed


|   iter    |  target   | batch_... | dropou... |  epochs   | learni... |   units   |
-------------------------------------------------------------------------------------
79/79 [==============================] - 0s 480us/step
| 1         | -9.498e-0 | 57.95     | 0.4803    | 34.64     | 0.003033  | 66.95     |
79/79 [==============================] - 0s 759us/step
| 2         | -8.986e-0 | 33.47     | 0.1232    | 37.32     | 0.003045  | 190.6     |
79/79 [==============================] - 0s 502us/step
| 3         | -8.327e-0 | 18.31     | 0.488     | 36.65     | 0.00114   | 72.73     |
79/79 [==============================] - 1s 573us/step
| 4         | -7.678e-0 | 36.54     | 0.2217    | 30.5      | 0.002217  | 97.24     |
79/79 [==============================] - 0s 699us/step
| 5         | -0.000204 | 84.53     | 0.1558    | 25.84     | 0.001895  | 134.2     |
79/79 [==============================] - 0s 465us/step
| 6         | -8.818e-0 | 103.9     | 0.1799    | 30.28     | 0.0030

79/79 [==============================] - 0s 950us/step
| 47        | -0.000106 | 93.51     | 0.4519    | 20.76     | 0.002354  | 193.9     |
79/79 [==============================] - 1s 1ms/step
| 48        | -5.391e-0 | 25.19     | 0.3236    | 36.84     | 0.0006919 | 208.2     |
79/79 [==============================] - 1s 1ms/step
| 49        | -2.747e-0 | 28.51     | 0.1407    | 37.47     | 0.001483  | 207.8     |
79/79 [==============================] - 1s 1ms/step
| 50        | -4.274e-0 | 102.8     | 0.1242    | 29.02     | 0.002745  | 199.5     |
79/79 [==============================] - 0s 940us/step
| 51        | -0.000118 | 106.4     | 0.4949    | 33.21     | 0.001298  | 194.9     |
79/79 [==============================] - 1s 3ms/step
| 52        | -3.229e-0 | 74.88     | 0.1701    | 31.15     | 0.001476  | 256.0     |
79/79 [==============================] - 0s 998us/step
| 53        | -0.000102 | 96.56     | 0.2641    | 29.48     | 0.004208  | 203.2     |
79/79 [==============

/var/folders/14/ljpzpqp91g1dpqy4sw00h0vm0000gn/T/ipykernel_37572/3556244187.py:99: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  naive_forecast = y_test.shift(1).fillna(method='bfill')


...

[*********************100%%**********************]  1 of 1 completed


|   iter    |  target   | batch_... | dropou... |  epochs   | learni... |   units   |
-------------------------------------------------------------------------------------
79/79 [==============================] - 1s 553us/step
| 1         | -0.000106 | 57.95     | 0.4803    | 34.64     | 0.003033  | 66.95     |
79/79 [==============================] - 1s 747us/step
| 2         | -0.000310 | 33.47     | 0.1232    | 37.32     | 0.003045  | 190.6     |
79/79 [==============================] - 0s 493us/step
| 3         | -0.000125 | 18.31     | 0.488     | 36.65     | 0.00114   | 72.73     |
79/79 [==============================] - 0s 522us/step
| 4         | -0.000148 | 36.54     | 0.2217    | 30.5      | 0.002217  | 97.24     |
79/79 [==============================] - 1s 663us/step
| 5         | -0.000107 | 84.53     | 0.1558    | 25.84     | 0.001895  | 134.2     |
79/79 [==============================] - 0s 520us/step
| 6         | -0.000189 | 103.9     | 0.1799    | 30.28     | 0.0030

79/79 [==============================] - 1s 1ms/step
| 47        | -0.000228 | 84.15     | 0.208     | 22.62     | 0.004791  | 247.0     |
79/79 [==============================] - 0s 611us/step
| 48        | -7.455e-0 | 84.09     | 0.3736    | 25.28     | 0.003866  | 131.7     |
79/79 [==============================] - 1s 679us/step
| 49        | -0.000133 | 53.45     | 0.4442    | 33.35     | 0.003054  | 136.4     |
79/79 [==============================] - 0s 600us/step
| 50        | -0.001073 | 84.34     | 0.1266    | 26.83     | 0.0006304 | 132.7     |
79/79 [==============================] - 1s 954us/step
| 51        | -0.000229 | 84.45     | 0.5       | 21.4      | 0.005     | 247.4     |
79/79 [==============================] - 1s 1ms/step
| 52        | -4.704e-0 | 84.23     | 0.4069    | 20.01     | 0.001724  | 249.6     |
79/79 [==============================] - 1s 969us/step
| 53        | -0.000262 | 84.61     | 0.1       | 21.42     | 0.0001    | 249.7     |
79/79 [==========

/var/folders/14/ljpzpqp91g1dpqy4sw00h0vm0000gn/T/ipykernel_37572/3556244187.py:99: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  naive_forecast = y_test.shift(1).fillna(method='bfill')


1/1 [==============================] - 0s 9ms/step


In [11]:
def print_results_with_metrics(results):
    """
    Print out results (model summary, evaluation metrics, and some forecast predictions)
    from the results dictionary returned by combined_workflow.
    """
    for ticker, data in results.items():
        print(f"Ticker: {ticker}")
        print("=" * 50)
        
        # 1. Print model summary (if the model is a statsmodels object)
        if "model" in data and hasattr(data["model"], "summary"):
            print("Model Summary:")
            print(data["model"].summary())
            print("\n")
        else:
            print("No statsmodels summary available for this model.\n")
        
        # 2. Print evaluation metrics (stored in "model results")
        if "model results" in data:
            metrics_dict = data["model results"]
            mae = metrics_dict.get("mae", None)
            mase = metrics_dict.get("mase", None)
            
            print("Evaluation Metrics:")
            print(f"Mean Absolute Error (MAE): {mae}")
            print(f"Mean Absolute Scaled Error (MASE): {mase}")
            print("\n")
        else:
            print("No evaluation metrics found.\n")

        # 3. Print monthly forecast predictions
        print("Monthly Forecast Predictions (first 5 values for each period):")
        for i_m in range(1, 13):  # Up to 12 months
            key_m = f"forecast_predictions_df_{i_m}m"
            if key_m in data:
                preds_array = data[key_m]
                print(f"{key_m}: {preds_array[:5]}")  # Print first 5 predictions
        print("\n")

        # 4. Print weekly forecast predictions (first 5 values)
        print("Weekly Forecast Predictions (first 5 values for each period):")
        for i_w in range(1, 53):  # Adjusted for 52 weeks
            key_w = f"forecast_predictions_df_{i_w}w"
            if key_w in data:
                preds_array = data[key_w]
                print(f"{key_w}: {preds_array[:5]}")  # Print first 5 predictions
        print("\n")
        
        print("=" * 50)
        print("\n")

print_results_with_metrics(results)

Ticker: SMH
No statsmodels summary available for this model.

Evaluation Metrics:
Mean Absolute Error (MAE): 0.00497982272626557
Mean Absolute Scaled Error (MASE): 0.25840614375336945


Monthly Forecast Predictions (first 5 values for each period):
forecast_predictions_df_1m: [-0.00603336  0.00350061  0.01073417  0.01584835  0.02505588]
forecast_predictions_df_2m: [0.01895928 0.02267959 0.02303164 0.01308429 0.02303035]
forecast_predictions_df_3m: [0.02675815 0.02179157 0.01123611 0.02323832 0.0250519 ]
forecast_predictions_df_4m: [0.01914166 0.00865985 0.01400372 0.00360714 0.01524917]
forecast_predictions_df_5m: [-0.00264038  0.01546542  0.01769909  0.01724158  0.00917826]
forecast_predictions_df_6m: [0.02081947 0.01497287 0.02703538 0.0144375  0.01661934]
forecast_predictions_df_7m: [0.01525775 0.01775421 0.02108803 0.01458285 0.01972709]
forecast_predictions_df_8m: [-0.02046833 -0.02111252 -0.01099344  0.00099403 -0.01176125]
forecast_predictions_df_9m: [-0.00785716  0.01485143  0.

In [12]:
lstm_metrics_clipboard={}
for ticker,data in results.items():
    if "model results" in data:
            metrics_dict = data["model results"]
            lstm_metrics_clipboard[ticker]=metrics_dict
            #mae = metrics_dict.get("mae", None)
            #mase = metrics_dict.get("mase", None)

for key,value in lstm_metrics_clipboard.items():
    print(key)
    print(value)

lstm_metrics_dataframe = pd.DataFrame(lstm_metrics_clipboard).T
lstm_metrics_dataframe.reset_index(inplace=True)
lstm_metrics_dataframe.rename(columns={'index':'ticker'}, inplace=True)
lstm_metrics_dataframe
lstm_metrics_dataframe.to_clipboard(index=False,sep=',')


SMH
{'mae': 0.00497982272626557, 'mase': 0.25840614375336945, 'time': 452.778137922287}
SOXX
{'mae': 0.008431454949973471, 'mase': 0.432345021145189, 'time': 403.19306778907776}
PSI
{'mae': 0.015480372260800242, 'mase': 0.758335653023237, 'time': 392.9798228740692}
XSD
{'mae': 0.012661567454724989, 'mase': 0.6065101378499113, 'time': 406.7711400985718}
IYW
{'mae': 0.005528212857372853, 'mase': 0.3710443876255269, 'time': 513.3658051490784}
XLK
{'mae': 0.005434786245362434, 'mase': 0.38318490459995536, 'time': 461.81556010246277}
VGT
{'mae': 0.02207959481612808, 'mase': 1.5282307316850496, 'time': 379.6760449409485}
FTEC
{'mae': 0.018143859754079287, 'mase': 1.2552191436418492, 'time': 291.6822597980499}
IGM
{'mae': 0.011534851919518184, 'mase': 0.7912380160754601, 'time': 429.4648139476776}
IXN
{'mae': 0.008791054778279262, 'mase': 0.6205005809957607, 'time': 478.30394172668457}


### Ratios


In [13]:
def calculate_sharpe_ratio(returns, annual_risk_free_rate=0.1,period='daily'):
    #excess_returns = rate_of_return(returns) - risk_free_rate
    
    # Convert annual risk-free rate to daily rate
    daily_risk_free_rate = (1 + annual_risk_free_rate) ** (1/252) - 1
    
    # Calculate mean daily log return
    mean_return = np.mean(returns)
    
    # Calculate excess daily log return
    excess_return = mean_return - daily_risk_free_rate
    
    # Calculate standard deviation of daily log returns
    std_return = np.std(returns)
    
    # Print diagnostic information
    #print(f"Mean Daily Log Return: {mean_return}")
    #print(f"Excess Daily Log Return: {excess_return}")
    #print(f"Standard Deviation of Daily Log Returns: {std_return}")
    
    # Check for zero standard deviation to avoid division by zero
    if std_return == 0:
        return 0
    
    # Calculate Sharpe ratio
    sharpe_ratio = (excess_return / std_return) * np.sqrt(252)  # Annualize the Sharpe ratio
    return sharpe_ratio
    #return excess_returns / np.std(returns)


def calculate_rachev_ratio(returns, lower_percentile=5, upper_percentile=95):
    # Step 1: Sort the returns
    sorted_returns = np.sort(returns)
    
    # Step 2: Determine the percentiles
    lower_threshold = np.percentile(sorted_returns, lower_percentile)
    upper_threshold = np.percentile(sorted_returns, upper_percentile)
    
    # Step 3: Calculate Expected Shortfall (ES)
    es = np.mean(sorted_returns[sorted_returns <= lower_threshold])
    
    # Step 4: Calculate Expected Gain (EG)
    eg = np.mean(sorted_returns[sorted_returns >= upper_threshold])
    
    # Step 5: Compute the Rachev Ratio
    rachev_ratio = eg / -es
    return rachev_ratio


"""def calculate_volatility_clustering(returns):
    squared_returns = returns ** 2
    n = len(squared_returns)
    
    # Mean of squared returns
    mean_squared_returns = np.mean(squared_returns)
    
    # Calculate the numerator and denominator for autocorrelation at lag 1
    numerator = np.sum((squared_returns[:-1] - mean_squared_returns) * (squared_returns[1:] - mean_squared_returns))
    denominator = np.sum((squared_returns - mean_squared_returns) ** 2)
    
    if denominator == 0:
        return 0  # Avoid division by zero
    
    rho_1 = numerator / denominator
    return rho_1"""
def calculate_volatility_clustering(returns):
    # Ensure returns is a NumPy array
    returns = np.array(returns)
    squared_returns = returns ** 2
    n = len(squared_returns)

    # Mean of squared returns
    mean_squared = np.mean(squared_returns)
    clustering = np.sum((squared_returns - mean_squared) ** 2) / (n - 1 if n > 1 else 1)
    return clustering

def calculate_sortino_ratio(log_returns, target_log_return=0.0):
    """
    Calculate the Sortino Ratio using log returns.
    
    Parameters:
    - log_returns (array-like): Array or list of log returns for the period.
    - target_log_return (float): The target log return. Default is 0, which is often used as a benchmark.
    
    Returns:
    - float: The Sortino Ratio.
    """
    # Convert input to a NumPy array for easier calculations
    log_returns = np.array(log_returns)
    
    # Calculate the average period log return (R)
    avg_log_return = np.mean(log_returns)
    
    # Calculate the Target Downside Deviation (TDD)
    downside_deviation = np.sqrt(
        np.mean(np.square(np.maximum(0, target_log_return - log_returns)))
    )
    
    # Calculate Sortino Ratio
    #sortino_ratio_value = (avg_log_return - target_log_return) / downside_deviation
    epsilon = 1e-8
    
    # Add epsilon to downside_deviation to prevent division by zero
    #if downside_deviation == 0:
     #   return np.nan  # Return NaN if downside deviation is zero
    
    # Calculate Sortino Ratio
    sortino_ratio_value = (avg_log_return - target_log_return) / (downside_deviation + epsilon)
    
    return sortino_ratio_value


In [14]:

def calculate_composite_score(
    forecasted_values, risk_percentage, rachev_ratio, sharpe_ratio, sortino_ratio, volatility_clustering, 
    mean_forecast, std_forecast, mean_rachev, std_rachev, mean_sharpe, std_sharpe, mean_sortino, std_sortino, 
    mean_volatility_clustering, std_volatility_clustering
):
    epsilon = 1e-8  # To prevent division by zero
    forecasted_mean = np.mean(forecasted_values)

    # Normalize the components with epsilon
    forecasted_mean_normalized = (forecasted_mean - mean_forecast) / (std_forecast + epsilon)
    rachev_normalized = (rachev_ratio - mean_rachev) / (std_rachev + epsilon)
    sharpe_normalized = (sharpe_ratio - mean_sharpe) / (std_sharpe + epsilon)
    sortino_normalized = (sortino_ratio - mean_sortino) / (std_sortino + epsilon)
    volatility_clustering_normalized = (volatility_clustering - mean_volatility_clustering) / (std_volatility_clustering + epsilon)

    # Composite score calculation
    score = (
        forecasted_mean_normalized
        - (risk_percentage * rachev_normalized)
        + sharpe_normalized
        + sortino_normalized
        - volatility_clustering_normalized
    )

    return score

def calculate_smoothing(data, alpha=0.1):
    if len(data) == 0:  # Explicitly check if the data is empty
        return []
    smoothed_data = []
    previous_value = data[0]
    for value in data:
        smoothed_value = alpha * value + (1 - alpha) * previous_value
        smoothed_data.append(smoothed_value)
        previous_value = smoothed_value
    return smoothed_data



def process_etf_data_weekly(tickers, etf_dict, smoothing=True):
    etf_pred_dict = {}

    # Determine the forecast periods dynamically from the etf_dict
    sample_etf = next(iter(etf_dict.values()))
    forecast_periods = [key.split('_')[-1] for key in sample_etf.keys() if key.startswith('forecast_predictions_df')]

    for etf_name in tickers:
        etf_pred_dict[etf_name] = {
            f"returns_{period}": etf_dict[etf_name][f"forecast_predictions_df_{period}"]
            for period in forecast_periods
        }

        for period in forecast_periods:
            returns = etf_pred_dict[etf_name][f"returns_{period}"]

            if smoothing:
                # Apply smoothing to returns
                smoothed_returns = calculate_smoothing(returns)
                etf_pred_dict[etf_name][f"smoothed_returns_{period}"] = smoothed_returns

                # Calculate metrics using smoothed returns
                etf_pred_dict[etf_name][f"rachev_ratio_{period}"] = calculate_rachev_ratio(smoothed_returns)
                etf_pred_dict[etf_name][f"sharpe_ratio_{period}"] = calculate_sharpe_ratio(smoothed_returns)
                etf_pred_dict[etf_name][f"sortino_ratio_{period}"] = calculate_sortino_ratio(smoothed_returns)
                etf_pred_dict[etf_name][f"volatility_clustering_{period}"] = calculate_volatility_clustering(smoothed_returns)
            else:
                # Calculate metrics using raw returns
                etf_pred_dict[etf_name][f"rachev_ratio_{period}"] = calculate_rachev_ratio(returns)
                etf_pred_dict[etf_name][f"sharpe_ratio_{period}"] = calculate_sharpe_ratio(returns)
                etf_pred_dict[etf_name][f"sortino_ratio_{period}"] = calculate_sortino_ratio(returns)
                etf_pred_dict[etf_name][f"volatility_clustering_{period}"] = calculate_volatility_clustering(returns)

    return etf_pred_dict


def calculate_means_and_stds_weekly(etf_pred_dict, forecast_period):
    returns_list = [etf_pred_dict[etf][f'returns_{forecast_period}'] for etf in etf_pred_dict]

    # Compute global means and standard deviations
    mean_forecast = np.mean([np.mean(returns) for returns in returns_list])
    std_forecast = np.std([np.mean(returns) for returns in returns_list])

    rachev_ratios = np.array([etf_pred_dict[etf][f'rachev_ratio_{forecast_period}'] for etf in etf_pred_dict])
    mean_rachev = np.mean(rachev_ratios)
    std_rachev = np.std(rachev_ratios)

    sharpe_ratios = np.array([etf_pred_dict[etf][f'sharpe_ratio_{forecast_period}'] for etf in etf_pred_dict])
    mean_sharpe = np.mean(sharpe_ratios)
    std_sharpe = np.std(sharpe_ratios)

    sortino_ratios = np.array([etf_pred_dict[etf][f'sortino_ratio_{forecast_period}'] for etf in etf_pred_dict])
    mean_sortino = np.mean(sortino_ratios)
    std_sortino = np.std(sortino_ratios)

    volatility_clustering = np.array([etf_pred_dict[etf][f'volatility_clustering_{forecast_period}'] for etf in etf_pred_dict])
    mean_volatility_clustering = np.mean(volatility_clustering)
    std_volatility_clustering = np.std(volatility_clustering)

    return (
        mean_forecast, std_forecast, mean_rachev, std_rachev,
        mean_sharpe, std_sharpe, mean_sortino, std_sortino,
        mean_volatility_clustering, std_volatility_clustering
    )


def calculate_scores_for_etfs_weekly(etf_pred_dict, forecast_period, risk_percentage):
    (mean_forecast, std_forecast, mean_rachev, std_rachev, 
     mean_sharpe, std_sharpe, mean_sortino, std_sortino, 
     mean_volatility_clustering, std_volatility_clustering) = calculate_means_and_stds_weekly(etf_pred_dict, forecast_period)

    scores = []
    for etf in etf_pred_dict:
        forecasted_values = etf_pred_dict[etf][f'returns_{forecast_period}']
        rachev_ratio = etf_pred_dict[etf][f'rachev_ratio_{forecast_period}']
        sharpe_ratio = etf_pred_dict[etf][f'sharpe_ratio_{forecast_period}']
        volatility_clustering = etf_pred_dict[etf][f'volatility_clustering_{forecast_period}']
        sortino_ratio = etf_pred_dict[etf][f'sortino_ratio_{forecast_period}']

        # Calculate the composite score
        score = calculate_composite_score(
            forecasted_values, risk_percentage, rachev_ratio, sharpe_ratio, 
            sortino_ratio, volatility_clustering, mean_forecast, std_forecast, 
            mean_rachev, std_rachev, mean_sharpe, std_sharpe, mean_sortino, 
            std_sortino, mean_volatility_clustering, std_volatility_clustering
        )

        scores.append({
            'ETF': etf,
            'Week': forecast_period,
            'RiskPercentage': risk_percentage,
            'Score': score
        })

    return scores


def select_top_etfs_weekly(df_scores, forecast_period):
    if df_scores.empty:
        print(f"No scores available for {forecast_period}. Skipping.")
        return []
    print(f"Processing scores for {forecast_period}:")
    print(df_scores.head())  # Check the top rows of the DataFrame
    top_etfs = df_scores.nlargest(2, 'Score')
    print(f"Top ETFs for {forecast_period}: {top_etfs['ETF'].tolist()}")
    return top_etfs['ETF'].tolist()


def generate_week_ranges(start_date, end_date):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    week_ranges = []

    while start < end:
        week_start = start
        week_end = start + timedelta(days=6)
        if week_end > end:
            week_end = end
        week_ranges.append((week_start.strftime('%Y-%m-%d'), week_end.strftime('%Y-%m-%d')))
        start += timedelta(days=7)

    return week_ranges




def gather_etf_data_for_weeks(selected_tickers, week_ranges):
    # Download all data for each ticker once
    print("Fetching full-year data for all tickers...")
    full_year_data = {}
    for ticker in selected_tickers:
        print(f"Downloading data for {ticker} for the entire year...")
        full_year_data[ticker] = yf.download(ticker, start=week_ranges[0][0], end=week_ranges[-1][1], progress=False)

    # Slice data into weekly ranges
    etf_histories = {}
    for start_date, end_date in week_ranges:
        week = f"{start_date} to {end_date}"
        etf_histories[week] = {}
        for ticker in selected_tickers:
            if ticker in full_year_data and not full_year_data[ticker].empty:
                # Slice the full-year data for the given week
                weekly_data = full_year_data[ticker].loc[start_date:end_date]
                if not weekly_data.empty:
                    etf_histories[week][ticker] = weekly_data
                else:
                    print(f"No data found for {ticker} in {week}")
            else:
                print(f"No data available for {ticker} in the full-year download.")
    return etf_histories




def initialize_shares_for_first_week(top_etfs_1w, etf_histories, week, investment_amount=50000):
    ticker_shares = {}
    first_trading_day_start = week.split(" to ")[0]

    for ticker in top_etfs_1w:
        etf_history = etf_histories.get(week, {}).get(ticker)

        if etf_history is not None:
            first_trading_day = pd.to_datetime(first_trading_day_start) + BDay(1)
            if first_trading_day not in etf_history.index:
                first_trading_day = etf_history.index[etf_history.index.searchsorted(first_trading_day)]

            price_on_first_trading_day = etf_history.loc[first_trading_day, 'Close']
            num_shares = (investment_amount * 0.9975) / price_on_first_trading_day
            ticker_shares[ticker] = num_shares
        else:
            print(f"No data found for {ticker} in {week}")
    return ticker_shares


def manage_etf_portfolio_weekly(
    top_etfs_previous, top_etfs_current, previous_week, current_week, ticker_shares, gathered_data_per_week
):
    etf_histories_for_current_week = gathered_data_per_week.get(current_week, {})
    top2etfs_previous = list(top_etfs_previous)
    top2etfs_current = list(top_etfs_current)

    print(f"\nTop 2 ETFs for {previous_week}: {top2etfs_previous}")
    print(f"Top 2 ETFs for {current_week}: {top2etfs_current}")

    etfs_to_sell = [etf for etf in top2etfs_previous if etf not in top2etfs_current]
    etfs_to_buy = [etf for etf in top2etfs_current if etf not in top2etfs_previous]

    # Ensure one-to-one mapping between sell and buy ETFs
    if len(etfs_to_sell) != len(etfs_to_buy):
        print("Mismatch between ETFs to sell and buy. Adjusting allocation...")
        return ticker_shares  # Abort if mismatched for now, you can implement custom logic

    # Allocate funds ETF-by-ETF
    for etf_sell, etf_buy in zip(etfs_to_sell, etfs_to_buy):
        no_of_shares = ticker_shares.get(etf_sell, 0)
        if no_of_shares > 0:
            # Selling old ETF
            if etf_sell in etf_histories_for_current_week:
                first_trading_day_sell_price = etf_histories_for_current_week[etf_sell].loc[
                    etf_histories_for_current_week[etf_sell].index[0], 'Close'
                ]
                selling_value = no_of_shares * first_trading_day_sell_price * 0.9975
                print(f"Sell {etf_sell}: {no_of_shares:.2f} shares at {first_trading_day_sell_price:.2f}. Total value: {selling_value:.2f}")

                # Remove sold ETF from portfolio
                del ticker_shares[etf_sell]

                # Buying new ETF
                if etf_buy in etf_histories_for_current_week:
                    first_trading_day_buy_price = etf_histories_for_current_week[etf_buy].loc[
                        etf_histories_for_current_week[etf_buy].index[0], 'Close'
                    ]
                    new_shares = (selling_value * 0.9975) / first_trading_day_buy_price
                    print(f"Buy {etf_buy}: {new_shares:.2f} shares at {first_trading_day_buy_price:.2f}.")
                    ticker_shares[etf_buy] = new_shares
                else:
                    print(f"Data for {etf_buy} is missing for {current_week}. Skipping purchase.")
            else:
                print(f"Data for {etf_sell} is missing for {current_week}. Skipping sale.")
        else:
            print(f"No shares found for {etf_sell} to sell.")

    print(f"Updated ticker shares after {current_week}: {ticker_shares}")
    return ticker_shares

def main_weekly(tickers, etf_dict, smoothing=True):
    etf_pred_dict = process_etf_data_weekly(tickers, etf_dict, smoothing=smoothing)

    risk_percentage = 0.10
    weekly_scores = {}

    # Calculate scores for all weeks (1 to 52)
    for week in range(1, 53):  # Updated range for 52 weeks
        week_key = f"{week}w"
        if any(f"returns_{week_key}" in etf_pred_dict[etf] for etf in etf_pred_dict):
            scores = calculate_scores_for_etfs_weekly(etf_pred_dict, week_key, risk_percentage)
            weekly_scores[week_key] = scores

    # Convert scores to DataFrames
    weekly_scores_dfs = {}
    for week_key, scores in weekly_scores.items():
        weekly_scores_dfs[week_key] = pd.DataFrame(scores)

    # Select top ETFs for each week
    top_etfs_weekly = {}
    for week_key, df_scores in weekly_scores_dfs.items():
        top_etfs_weekly[week_key] = df_scores.nlargest(2, 'Score')

    # Generate week ranges for ETF data gathering
    week_ranges = generate_week_ranges('2024-01-01', '2024-12-31')  # Full year for 52 weeks

    # Gather ETF historical data
    etf_histories = gather_etf_data_for_weeks(tickers, week_ranges)

    # Map week keys to date ranges
    week_key_mapping = {f"{i+1}w": week_range for i, week_range in enumerate(etf_histories.keys())}

    # Align top ETFs with historical data
    aligned_top_etfs_weekly = {}
    for week_key, df_scores in weekly_scores_dfs.items():
        forecast_period = week_key_mapping.get(week_key, None)
        if forecast_period:
            aligned_top_etfs_weekly[forecast_period] = select_top_etfs_weekly(df_scores, forecast_period)

    # Initialize and manage portfolio
    ticker_shares = {}
    ticker_shares_per_week = {}
    for i, week_range in enumerate(etf_histories.keys()):
        current_week_key = week_range
        if i == 0:
            # Initialize shares for the first week
            ticker_shares = initialize_shares_for_first_week(
                aligned_top_etfs_weekly.get(current_week_key, []),  # Fallback to empty list if key is missing
                etf_histories,
                current_week_key
            )
            ticker_shares_per_week[current_week_key] = ticker_shares.copy()
        else:
            # Manage portfolio for subsequent weeks
            prev_week_key = list(etf_histories.keys())[i - 1]
            if current_week_key not in aligned_top_etfs_weekly:
                print(f"Warning: Missing alignment for {current_week_key}. Skipping week.")
                continue

            ticker_shares = manage_etf_portfolio_weekly(
                aligned_top_etfs_weekly.get(prev_week_key, []),  # Fallback to empty list
                aligned_top_etfs_weekly[current_week_key],
                prev_week_key,
                current_week_key,
                ticker_shares,
                etf_histories
            )
            ticker_shares_per_week[current_week_key] = ticker_shares.copy()


    # Define the first trading day of the 53rd week
    first_trading_day_53w = '2025-01-01'

    # Identify the 52nd week date ranges
    week_52_range = list(ticker_shares_per_week.keys())[-1]
    week_53_start = first_trading_day_53w
    print(f"Using data for the 52nd week: {week_52_range}")
    print(f"Fetching data starting from the first trading day of the 53rd week: {week_53_start}")

    week_53_end = '2025-01-06'
    etf_values_53w = {}

    # Ensure 52nd week data exists
    if week_52_range in ticker_shares_per_week:
        ticker_shares_52w = ticker_shares_per_week[week_52_range]

        # Fetch the first trading day price of the 53rd week for each ETF
        for ticker, shares in ticker_shares_52w.items():
            print(f"Fetching data for ticker {ticker} starting from {week_53_start}...")
            data = yf.download(ticker, start=week_53_start, end=week_53_end)

            if not data.empty:
                closing_price_53w = data['Close'].iloc[0]
                total_value = shares * closing_price_53w
                etf_values_53w[ticker] = total_value
                print(f"{ticker}: {shares:.2f} shares at ${closing_price_53w:.2f} each, total value: ${total_value:.2f}")
            else:
                print(f"{ticker}: No data available for the 53rd week's first trading day.")
    else:
        print(f"No data available in ticker_shares_per_week for the 52nd week: {week_52_range}")

    # Display total portfolio value for the 53rd week's first trading day
    if etf_values_53w:
        print("\nETF values on the 53rd week's first trading day:")
        total_value = sum(etf_values_53w.values())
        print(f"Total portfolio value: {total_value:.2f}")
        for ticker, value in etf_values_53w.items():
            print(f"{ticker}: {value:.2f}")
    else:
        print("No values could be calculated for the 53rd week's first trading day.")

    return ticker_shares_per_week



In [15]:
# Now call the weekly workflow using the 'results' as etf_dict
ticker_shares_per_week_wo_smoothing = main_weekly(tickers, results, smoothing=False)


Fetching full-year data for all tickers...


[*********************100%%**********************]  1 of 1 completed

Processing scores for 2024-01-01 to 2024-01-07:
    ETF Week  RiskPercentage     Score
0   SMH   1w             0.1  0.216696
1  SOXX   1w             0.1  2.725782
2   PSI   1w             0.1 -3.799003
3   XSD   1w             0.1  4.355461
4   IYW   1w             0.1  1.110150
Top ETFs for 2024-01-01 to 2024-01-07: ['XSD', 'VGT']
Processing scores for 2024-01-08 to 2024-01-14:
    ETF Week  RiskPercentage     Score
0   SMH   2w             0.1  1.068845
1  SOXX   2w             0.1  2.726852
2   PSI   2w             0.1 -2.891139
3   XSD   2w             0.1  1.288767
4   IYW   2w             0.1  1.024286
Top ETFs for 2024-01-08 to 2024-01-14: ['SOXX', 'IGM']
Processing scores for 2024-01-15 to 2024-01-21:
    ETF Week  RiskPercentage     Score
0   SMH   3w             0.1  1.427721
1  SOXX   3w             0.1  2.819627
2   PSI   3w             0.1 -4.237536
3   XSD   3w             0.1  1.819101
4   IYW   3w             0.1  0.770994
Top ETFs for 2024-01-15 to 2024-01-21: ['SOXX


[*********************100%%**********************]  1 of 1 completed

SOXX: 251.51 shares at $216.82 each, total value: $54532.43

ETF values on the 53rd week's first trading day:
Total portfolio value: 114800.08
VGT: 60267.64
SOXX: 54532.43


In [16]:
ticker_shares_per_week_with_smoothing = main_weekly(tickers, results, smoothing=True)


Fetching full-year data for all tickers...
Processing scores for 2024-01-01 to 2024-01-07:
    ETF Week  RiskPercentage     Score
0   SMH   1w             0.1 -0.288996
1  SOXX   1w             0.1  1.867843
2   PSI   1w             0.1 -3.833493
3   XSD   1w             0.1  5.355905
4   IYW   1w             0.1  0.154987
Top ETFs for 2024-01-01 to 2024-01-07: ['XSD', 'VGT']
Processing scores for 2024-01-08 to 2024-01-14:
    ETF Week  RiskPercentage     Score
0   SMH   2w             0.1  1.029279
1  SOXX   2w             0.1  2.490258
2   PSI   2w             0.1 -2.914628
3   XSD   2w             0.1  1.598692
4   IYW   2w             0.1  0.606122
Top ETFs for 2024-01-08 to 2024-01-14: ['VGT', 'SOXX']
Processing scores for 2024-01-15 to 2024-01-21:
    ETF Week  RiskPercentage     Score
0   SMH   3w             0.1  1.264296
1  SOXX   3w             0.1  3.041080
2   PSI   3w             0.1 -4.568948
3   XSD   3w             0.1  1.732651
4   IYW   3w             0.1  0.016716
To

[*********************100%%**********************]  1 of 1 completed

VGT: 101.58 shares at $621.34 each, total value: $63116.76
Fetching data for ticker XSD starting from 2025-01-01...



[*********************100%%**********************]  1 of 1 completed

XSD: 212.22 shares at $252.06 each, total value: $53492.12

ETF values on the 53rd week's first trading day:
Total portfolio value: 116608.88
VGT: 63116.76
XSD: 53492.12


In [17]:
def process_etf_data_monthly(tickers, etf_dict, smoothing=True):
    etf_pred_dict = {}

    # Determine forecast periods dynamically from the etf_dict
    sample_etf = next(iter(etf_dict.values()))
    forecast_periods = [key.split('_')[-1] for key in sample_etf.keys() if key.startswith('forecast_predictions_df')]

    for etf_name in tickers:
        etf_pred_dict[etf_name] = {
            f"returns_{period}": etf_dict[etf_name][f"forecast_predictions_df_{period}"]
            for period in forecast_periods
        }

        for period in forecast_periods:
            returns = etf_pred_dict[etf_name][f"returns_{period}"]

            if smoothing:
                # Apply smoothing
                smoothed_returns = calculate_smoothing(returns)
                etf_pred_dict[etf_name][f"smoothed_returns_{period}"] = smoothed_returns

                # Calculate metrics using smoothed returns
                etf_pred_dict[etf_name][f"rachev_ratio_{period}"] = calculate_rachev_ratio(smoothed_returns)
                etf_pred_dict[etf_name][f"sharpe_ratio_{period}"] = calculate_sharpe_ratio(smoothed_returns)
                etf_pred_dict[etf_name][f"sortino_ratio_{period}"] = calculate_sortino_ratio(smoothed_returns)
                etf_pred_dict[etf_name][f"volatility_clustering_{period}"] = calculate_volatility_clustering(smoothed_returns)
            else:
                # Calculate metrics using raw returns
                etf_pred_dict[etf_name][f"rachev_ratio_{period}"] = calculate_rachev_ratio(returns)
                etf_pred_dict[etf_name][f"sharpe_ratio_{period}"] = calculate_sharpe_ratio(returns)
                etf_pred_dict[etf_name][f"sortino_ratio_{period}"] = calculate_sortino_ratio(returns)
                etf_pred_dict[etf_name][f"volatility_clustering_{period}"] = calculate_volatility_clustering(returns)

    return etf_pred_dict


def calculate_means_and_stds_monthly(etf_pred_dict, forecast_period):
    returns_list = [etf_pred_dict[etf][f'returns_{forecast_period}'] for etf in etf_pred_dict]

    # Compute global means and standard deviations
    mean_forecast = np.mean([np.mean(returns) for returns in returns_list])
    std_forecast = np.std([np.mean(returns) for returns in returns_list])

    print(f"\nDebug: Forecast Period = {forecast_period}")
    print(f"All Returns Means: {[np.mean(returns) for returns in returns_list]}")
    print(f"Mean Forecast = {mean_forecast}, Std Forecast = {std_forecast}")

    rachev_ratios = np.array([etf_pred_dict[etf][f'rachev_ratio_{forecast_period}'] for etf in etf_pred_dict])
    print(f"All Rachev Ratios: {rachev_ratios}")
    mean_rachev = np.mean(rachev_ratios)
    std_rachev = np.std(rachev_ratios)

    sharpe_ratios = np.array([etf_pred_dict[etf][f'sharpe_ratio_{forecast_period}'] for etf in etf_pred_dict])
    print(f"All Sharpe Ratios: {sharpe_ratios}")
    mean_sharpe = np.mean(sharpe_ratios)
    std_sharpe = np.std(sharpe_ratios)

    sortino_ratios = np.array([etf_pred_dict[etf][f'sortino_ratio_{forecast_period}'] for etf in etf_pred_dict])
    print(f"All Sortino Ratios: {sortino_ratios}")
    mean_sortino = np.mean(sortino_ratios)
    std_sortino = np.std(sortino_ratios)

    volatility_clustering = np.array([etf_pred_dict[etf][f'volatility_clustering_{forecast_period}'] for etf in etf_pred_dict])
    print(f"All Volatility Clustering: {volatility_clustering}")
    mean_volatility_clustering = np.mean(volatility_clustering)
    std_volatility_clustering = np.std(volatility_clustering)

    print(f"Mean Rachev = {mean_rachev}, Std Rachev = {std_rachev}")
    print(f"Mean Sharpe = {mean_sharpe}, Std Sharpe = {std_sharpe}")
    print(f"Mean Sortino = {mean_sortino}, Std Sortino = {std_sortino}")
    print(f"Mean Volatility Clustering = {mean_volatility_clustering}, Std Volatility Clustering = {std_volatility_clustering}")

    return (
        mean_forecast, std_forecast, mean_rachev, std_rachev,
        mean_sharpe, std_sharpe, mean_sortino, std_sortino,
        mean_volatility_clustering, std_volatility_clustering
    )

def calculate_scores_for_etfs_monthly(etf_pred_dict, forecast_period, risk_percentage, smoothing = True):
    (mean_forecast, std_forecast, mean_rachev, std_rachev, 
     mean_sharpe, std_sharpe, mean_sortino, std_sortino, 
     mean_volatility_clustering, std_volatility_clustering) = calculate_means_and_stds_monthly(etf_pred_dict, forecast_period)

    scores = []
    for etf in etf_pred_dict:
        forecasted_values = etf_pred_dict[etf][f'returns_{forecast_period}']
        rachev_ratio = etf_pred_dict[etf][f'rachev_ratio_{forecast_period}']
        sharpe_ratio = etf_pred_dict[etf][f'sharpe_ratio_{forecast_period}']
        volatility_clustering = etf_pred_dict[etf][f'volatility_clustering_{forecast_period}']
        sortino_ratio = etf_pred_dict[etf][f'sortino_ratio_{forecast_period}']

        # Debugging: Log inputs to composite score calculation
        print(f"\nDebug: ETF = {etf}, Forecast Period = {forecast_period}")
        print(f"Forecasted Values Mean: {np.mean(forecasted_values)}")
        print(f"Rachev Ratio: {rachev_ratio}, Sharpe Ratio: {sharpe_ratio}")
        print(f"Sortino Ratio: {sortino_ratio}, Volatility Clustering: {volatility_clustering}")
        print(f"Means and Stds: Mean Forecast = {mean_forecast}, Std Forecast = {std_forecast}")
        
        # Calculate the composite score
        score = calculate_composite_score(
            forecasted_values, risk_percentage, rachev_ratio, sharpe_ratio, 
            sortino_ratio, volatility_clustering, mean_forecast, std_forecast, 
            mean_rachev, std_rachev, mean_sharpe, std_sharpe, mean_sortino, 
            std_sortino, mean_volatility_clustering, std_volatility_clustering
        )

        scores.append({
            'ETF': etf,
            'Month': forecast_period,
            'RiskPercentage': risk_percentage,
            'Score': score
        })

        # Debugging: Log the calculated score
        print(f"Calculated Score for {etf} ({forecast_period}): {score}")

    return scores

def select_top_etfs_monthly(df_scores, forecast_period):
    if df_scores.empty:
        print(f"No scores available for {forecast_period}. Skipping.")
        return []
    print(f"Processing scores for {forecast_period}:")
    print(df_scores.head())  # Check the top rows of the DataFrame
    top_etfs = df_scores.nlargest(2, 'Score')
    print(f"Top ETFs for {forecast_period}: {top_etfs['ETF'].tolist()}")
    return top_etfs['ETF'].tolist()


def generate_month_ranges(start_date, end_date):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    month_ranges = []

    while start < end:
        month_start = start
        month_end = (start + timedelta(days=31)).replace(day=1) - timedelta(days=1)
        if month_end > end:
            month_end = end
        month_ranges.append((month_start.strftime('%Y-%m-%d'), month_end.strftime('%Y-%m-%d')))
        start = month_end + timedelta(days=1)

    return month_ranges

def gather_etf_data_for_months(tickers, month_ranges):
    etf_histories = {}
    for start_date, end_date in month_ranges:
        month = f"{start_date} to {end_date}"
        etf_histories[month] = {}
        for ticker in tickers:
            etf_data = yf.download(ticker, start=start_date, end=end_date, progress=False)
            if etf_data.empty:
                print(f"No data found for {ticker} in {month}")
                continue
            etf_data.index = pd.to_datetime(etf_data.index)
            etf_histories[month][ticker] = etf_data
    return etf_histories

# Function to initialize shares for the first month
def initialize_shares_for_first_month(top_etfs_1m, etf_histories, month, investment_amount=50000):
    ticker_shares = {}
    first_trading_day_start = month.split(" to ")[0]

    for ticker in top_etfs_1m:
        etf_history = etf_histories.get(month, {}).get(ticker)

        if etf_history is not None:
            first_trading_day = pd.to_datetime(first_trading_day_start) + BDay(1)
            if first_trading_day not in etf_history.index:
                first_trading_day = etf_history.index[etf_history.index.searchsorted(first_trading_day)]

            price_on_first_trading_day = etf_history.loc[first_trading_day, 'Close']
            num_shares = (investment_amount * 0.9975) / price_on_first_trading_day
            print(f"Shares 1st month: ({investment_amount} * 0.9975) / {price_on_first_trading_day}")
            ticker_shares[ticker] = num_shares
            print(f"Ticker: {ticker}, First trading day: {first_trading_day.date()}, Price: {price_on_first_trading_day}, Shares: {num_shares:.2f}")
        else:
            print(f"No data found for {ticker} in {month}")
    return ticker_shares


def manage_etf_portfolio_monthly(
    top_etfs_previous, top_etfs_current, previous_month, current_month, ticker_shares, gathered_data_per_month
):
    etf_histories_for_current_month = gathered_data_per_month.get(current_month, {})

    # Extract ETFs as lists to preserve order
    top2etfs_previous = list(top_etfs_previous)
    top2etfs_current = list(top_etfs_current)

    print(f"Top 2 ETFs for {previous_month}: {top2etfs_previous}")
    print(f"Top 2 ETFs for {current_month}: {top2etfs_current}")

    # Step 1: Identify common ETFs (present in both months)
    etfs_common = [etf for etf in top2etfs_previous if etf in top2etfs_current]

    # Step 2: Identify ETFs to sell (present in the previous month but not in the current month)
    etfs_to_sell = [etf for etf in top2etfs_previous if etf not in top2etfs_current]

    # Step 3: Identify ETFs to buy (present in the current month but not in the previous month)
    etfs_to_buy = [etf for etf in top2etfs_current if etf not in top2etfs_previous]

    # Step 4: Sell ETFs that are no longer in the current top ETFs
    selling_values = {}  # Track selling values for each ETF being sold
    for etf in etfs_to_sell:
        no_of_shares = ticker_shares.get(etf, 0)
        if no_of_shares > 0:
            # Check if data for the ETF is available in the current month
            if etf in etf_histories_for_current_month:
                first_trading_day_current_month = etf_histories_for_current_month[etf].loc[
                    etf_histories_for_current_month[etf].index[0], 'Close'
                ]
                
                selling_value = no_of_shares * first_trading_day_current_month * 0.9975
                selling_values[etf] = selling_value  # Store selling value for this ETF
                print(f"Formula: {no_of_shares} * {first_trading_day_current_month} * 0.9975")
                print(f"Sell {etf}: {no_of_shares:.2f} shares at {first_trading_day_current_month:.2f}. Total value: {selling_value:.2f}\n")
                del ticker_shares[etf]
            else:
                print(f"Data for {etf} is missing for {current_month}. Skipping sale.")
        else:
            print(f"No shares found for {etf} to sell.")

    # Step 5: Buy new ETFs that were not in the previous month's top ETFs
    for etf_to_buy, etf_to_sell in zip(etfs_to_buy, etfs_to_sell):
        selling_value = selling_values.get(etf_to_sell, 0)
        if selling_value > 0 and etf_to_buy in etf_histories_for_current_month:
            first_trading_day_new_etf = etf_histories_for_current_month[etf_to_buy].loc[
                etf_histories_for_current_month[etf_to_buy].index[0], 'Close'
            ]
            new_shares = (selling_value * 0.9975) / first_trading_day_new_etf
            print(f"Formula: ({selling_value} * 0.9975) / {first_trading_day_new_etf}")
            print(f"Buy {etf_to_buy}: {new_shares:.2f} shares at {first_trading_day_new_etf:.2f}.\n")
            ticker_shares[etf_to_buy] = new_shares
        else:
            print(f"Data for {etf_to_buy} is missing or no selling value available. Skipping purchase of {etf_to_buy}.")

    # Step 6: Maintain the order of ETFs in the ticker_shares dictionary based on the current top 2 ETFs
    ordered_ticker_shares = {etf: ticker_shares[etf] for etf in top2etfs_current if etf in ticker_shares}

    print(f"Updated ticker shares after {current_month}: {ordered_ticker_shares}")
    print("")
    return ordered_ticker_shares



def main_monthly(tickers, etf_dict, smoothing=True):
    etf_pred_dict = process_etf_data_monthly(tickers, etf_dict, smoothing=smoothing)

    risk_percentage = 0.10
    monthly_scores = {}

    # Calculate scores for all months
    for month in range(1, 13):
        month_key = f"{month}m"
        if any(f"returns_{month_key}" in etf_pred_dict[etf] for etf in etf_pred_dict):
            scores = calculate_scores_for_etfs_weekly(etf_pred_dict, month_key, risk_percentage)  # Reusing weekly function
            monthly_scores[month_key] = scores

    # Convert scores to DataFrames
    monthly_scores_dfs = {}
    for month_key, scores in monthly_scores.items():
        monthly_scores_dfs[month_key] = pd.DataFrame(scores)

    # Select top ETFs for each month
    top_etfs_monthly = {}
    for month_key, df_scores in monthly_scores_dfs.items():
        top_etfs_monthly[month_key] = df_scores.nlargest(2, 'Score')

    # Generate month ranges for ETF data gathering
    month_ranges = generate_month_ranges('2024-01-01', '2025-01-01')

    # Gather ETF historical data
    etf_histories = gather_etf_data_for_weeks(tickers, month_ranges)

    # Map month keys to date ranges
    month_key_mapping = {f"{i+1}m": month_range for i, month_range in enumerate(etf_histories.keys())}

    # Align top ETFs with historical data
    aligned_top_etfs_monthly = {}
    for month_key, df_scores in monthly_scores_dfs.items():
        forecast_period = month_key_mapping.get(month_key, None)
        if forecast_period:
            aligned_top_etfs_monthly[forecast_period] = select_top_etfs_weekly(df_scores, forecast_period)

    # Initialize and manage portfolio
    ticker_shares = {}
    ticker_shares_per_month = {}
    for i, month_range in enumerate(etf_histories.keys()):
        current_month_key = month_range
        if i == 0:
            # Initialize shares for the first month
            ticker_shares = initialize_shares_for_first_week(
                aligned_top_etfs_monthly[current_month_key],
                etf_histories,
                current_month_key
            )
            ticker_shares_per_month[current_month_key] = ticker_shares.copy()
        else:
            # Manage portfolio for subsequent months
            prev_month_key = list(etf_histories.keys())[i - 1]
            ticker_shares = manage_etf_portfolio_monthly(
                aligned_top_etfs_monthly[prev_month_key],
                aligned_top_etfs_monthly[current_month_key],
                prev_month_key,
                current_month_key,
                ticker_shares,
                etf_histories
            )
            ticker_shares_per_month[current_month_key] = ticker_shares.copy()

    # Define the first trading day of the 13th month
    first_trading_day_13m = '2025-01-01'

    # Identify the 12th and 13th month date ranges
    month_12_range = list(ticker_shares_per_month.keys())[-1]
    month_13_start = first_trading_day_13m
    print(f"Using data for the 12th month: {month_12_range}")
    print(f"Fetching data starting from the first trading day of the 13th month: {month_13_start}")

    month_13_end = '2025-01-06'
    etf_values_13m = {}

    # Ensure 12th month data exists
    if month_12_range in ticker_shares_per_month:
        ticker_shares_12m = ticker_shares_per_month[month_12_range]

        # Fetch the first trading day price of the 13th month for each ETF
        for ticker, shares in ticker_shares_12m.items():
            print(f"Fetching data for ticker {ticker} starting from {month_13_start}...")
            data = yf.download(ticker, start=month_13_start, end=month_13_end)

            if not data.empty:
                closing_price_13m = data['Close'].iloc[0]
                total_value = shares * closing_price_13m
                etf_values_13m[ticker] = total_value
                print(f"{ticker}: {shares:.2f} shares at ${closing_price_13m:.2f} each, total value: ${total_value:.2f}")
            else:
                print(f"{ticker}: No data available for the 13th month's first trading day.")
    else:
        print(f"No data available in ticker_shares_per_month for the 12th month: {month_12_range}")

    # Display total portfolio value for the 13th month's first trading day
    if etf_values_13m:
        print("\nETF values on the 13th month's first trading day:")
        total_value = sum(etf_values_13m.values())
        print(f"Total portfolio value: {total_value:.2f}")
        for ticker, value in etf_values_13m.items():
            print(f"{ticker}: {value:.2f}")
    else:
        print("No values could be calculated for the 13th month's first trading day.")

    return ticker_shares_per_month



In [18]:
# Now call the weekly workflow using the 'results' as etf_dict
ticker_shares_per_month_wo_smoothing = main_monthly(tickers, results,False)


Fetching full-year data for all tickers...


[*********************100%%**********************]  1 of 1 completed

Processing scores for 2024-01-01 to 2024-01-31:
    ETF Week  RiskPercentage     Score
0   SMH   1m             0.1  0.033572
1  SOXX   1m             0.1  2.736625
2   PSI   1m             0.1 -4.277749
3   XSD   1m             0.1  2.725365
4   IYW   1m             0.1  1.193205
Top ETFs for 2024-01-01 to 2024-01-31: ['SOXX', 'XSD']
Processing scores for 2024-02-01 to 2024-02-29:
    ETF Week  RiskPercentage     Score
0   SMH   2m             0.1  1.404713
1  SOXX   2m             0.1  3.174479
2   PSI   2m             0.1 -4.642279
3   XSD   2m             0.1  2.599059
4   IYW   2m             0.1  0.559495
Top ETFs for 2024-02-01 to 2024-02-29: ['SOXX', 'XSD']
Processing scores for 2024-03-01 to 2024-03-31:
    ETF Week  RiskPercentage     Score
0   SMH   3m             0.1  1.421531
1  SOXX   3m             0.1  2.715397
2   PSI   3m             0.1 -4.134424
3   XSD   3m             0.1  2.000352
4   IYW   3m             0.1  0.985956
Top ETFs for 2024-03-01 to 2024-03-31: ['VGT


[*********************100%%**********************]  1 of 1 completed

SOXX: 231.78 shares at $216.82 each, total value: $50254.99

ETF values on the 13th month's first trading day:
Total portfolio value: 121075.05
XSD: 70820.06
SOXX: 50254.99


In [19]:
# Now call the weekly workflow using the 'results' as etf_dict
ticker_shares_per_month_with_smoothing = main_monthly(tickers, results,True)


Fetching full-year data for all tickers...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Processing scores for 2024-01-01 to 2024-01-31:
    ETF Week  RiskPercentage     Score
0   SMH   1m             0.1 -0.025904
1  SOXX   1m             0.1  2.003489
2   PSI   1m             0.1 -4.121598
3   XSD   1m             0.1  4.357780
4   IYW   1m             0.1  0.813923
Top ETFs for 2024-01-01 to 2024-01-31: ['XSD', 'SOXX']
Processing scores for 2024-02-01 to 2024-02-29:
    ETF Week  RiskPercentage     Score
0   SMH   2m             0.1  1.689090
1  SOXX   2m             0.1  2.954220
2   PSI   2m             0.1 -5.005730
3   XSD   2m             0.1  1.964066
4   IYW   2m             0.1  0.506176
Top ETFs for 2024-02-01 to 2024-02-29: ['SOXX', 'XSD']
Processing scores for 2024-03-01 to 2024-03-31:
    ETF Week  RiskPercentage     Score
0   SMH   3m             0.1  0.861789
1  SOXX   3m             0.1  2.480120
2   PSI   3m             0.1 -4.193231
3   XSD   3m             0.1  1.336093
4   IYW   3m             0.1  0.399672
Top ETFs for 2024-03-01 to 2024-03-31: ['VGT